In [145]:
import pandas as pd
import requests
import matplotlib.pyplot as plt
from google.transit import gtfs_realtime_pb2
from pprint import pprint

%matplotlib inline

Medium post on working with gtfs realtime data: https://medium.com/@BennettGarner/parsing-gtfs-format-transit-data-in-real-time-with-python-3a528ba7aab7

And the repo with actual code: https://github.com/bennett39/mta-real-time-data/blob/master/gtfs.py

GTFS reference material: https://developers.google.com/transit/gtfs-realtime/reference/

In [146]:
df_stops = pd.read_csv('data\google_transit\stops.csv')

In [147]:
api_key = '230ffa803afc2f7fa526a3e39982256d'

In [148]:
url_123456S = f'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=1'
url_ACEHS = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=26'
url_NQRW = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=16'
url_BDFM = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=21'
url_L = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=2'
url_G = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=31'
url_JZ = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=36'
url_7 = 'http://datamine.mta.info/mta_esi.php?key={api_key}&feed_id=51'

In [149]:
# list of unique URLs for each train line: http://datamine.mta.info/list-of-feeds
url = url_123456S

In [150]:
# make request
response = requests.get(url)
feed = gtfs_realtime_pb2.FeedMessage()
feed.ParseFromString(response.content)

87453

In [151]:
type(feed.entity)

google.protobuf.pyext._message.RepeatedCompositeContainer

In [152]:
feed.entity[0]

id: "000001"
trip_update {
  trip {
    trip_id: "066500_2..S12X333"
    start_date: "20190824"
    route_id: "2"
  }
  stop_time_update {
    arrival {
      time: 1566664600
    }
    stop_id: "247S"
  }
}

In [153]:
feed.entity[0].trip_update.stop_time_update[0].stop_id

'247S'

In [154]:
one_line = []
two_line = []
three_line = []
four_line = []
five_line = []
six_line = []
s_line = []

for item in feed.entity:
    #Get stop name (need to check that each field exists to avoid errors)
    if not item.HasField('trip_update'):
        continue
    else:
        stop = item.trip_update.stop_time_update[0].stop_id
    
    #Get train line
    if not item.HasField('trip_update'):
        continue
    else:
        line = item.trip_update.trip.route_id
    
    #Append stop name to correct list
    if line == '1':
        one_line.append(stop)
    elif line == '2':
        two_line.append(stop)
    elif line == '3':
        three_line.append(stop)
    elif line == '4':
        four_line.append(stop)
    elif line == '5':
        five_line.append(stop)
    elif line == '6':
        six_line.append(stop)
    elif line == 'S':
        s_line.append(stop)

In [155]:
one_line_update = []

for stop in one_line:
    station_name = df_stops[df_stops['stop_id'] == stop]['stop_name'].to_string(index=False)
    if station_name not in one_line_update:
        one_line_update.append(station_name)

In [156]:
one_line_update

['145 St',
 'Marble Hill - 225 St',
 '215 St',
 '168 St - Washington Hts',
 'Van Cortlandt Park - 242 St',
 '137 St - City College']

In [157]:
one_line_update = []

for stop in one_line:
    station_name = df_stops[df_stops['stop_id'] == stop]['stop_name'].to_string(index=False)
    one_line_update.append(station_name)

In [158]:
one_line_update

['145 St',
 'Marble Hill - 225 St',
 '215 St',
 '168 St - Washington Hts',
 'Van Cortlandt Park - 242 St',
 '145 St',
 'Van Cortlandt Park - 242 St',
 '137 St - City College',
 'Van Cortlandt Park - 242 St',
 '137 St - City College',
 'Van Cortlandt Park - 242 St']